# Лабораторная работа №3
## Задание:
1 . Провести классификацию найденного датасета, методами линейной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
#### Подключение библиотек


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from imblearn.under_sampling import RandomUnderSampler

### Загрузка данных, вывод датасета


In [2]:
Dataset = pd.read_csv('property_assessment.csv')
label_encoder = LabelEncoder()
Dataset['COND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])
Dataset['CITY_ENC'] = label_encoder.fit_transform(Dataset['CITY'])
Dataset['BLDG_TYPE_ENC'] = label_encoder.fit_transform(Dataset['BLDG_TYPE'])
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PID            10321 non-null  int64  
 1   CM_ID          10321 non-null  float64
 2   GIS_ID         10321 non-null  int64  
 3   ST_NUM         10321 non-null  float64
 4   ZIP_CODE       10321 non-null  float64
 5   BLDG_SEQ       10321 non-null  int64  
 6   NUM_BLDGS      10321 non-null  int64  
 7   LUC            10321 non-null  int64  
 8   RES_FLOOR      10321 non-null  float64
 9   GROSS_AREA     10321 non-null  float64
 10  LIVING_AREA    10321 non-null  float64
 11  LAND_VALUE     10321 non-null  int64  
 12  BLDG_VALUE     10321 non-null  int64  
 13  SFYI_VALUE     10321 non-null  int64  
 14  TOTAL_VALUE    10321 non-null  int64  
 15  YR_BUILT       10321 non-null  float64
 16  YR_REMODEL     10321 non-null  float64
 17  BED_RMS        10321 non-null  float64
 18  FULL_B

### Преобразование данных и разделение на признаки

In [3]:
X = Dataset[['BLDG_TYPE_ENC', 'LIVING_AREA', 'TOTAL_VALUE', 'CITY_ENC']]
y = Dataset['COND_ENC']

### Поиск лучшего параметра

In [4]:
X_train, X_test, y_train, y_test = train_test_split(Dataset.drop('COND_ENC', axis=1), Dataset['COND_ENC'], test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

under_sampler = RandomUnderSampler(random_state=42)

X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

print(pd.Series(y_resampled).value_counts())
print(y_test.unique())
# Создание модели
linear_model = LinearRegression()

# Подготовка параметров для GridSearchCV
linear_params = {'fit_intercept': [True, False]}

# Инициализация GridSearchCV
linear_grid = GridSearchCV(linear_model, linear_params, cv=5)

# Обучение модели
linear_grid.fit(X_resampled, y_resampled)

# Вывод результатов
print("Лучшие параметры для линейной регрессии:", linear_grid.best_params_)

COND_ENC
0    1
1    1
2    1
3    1
4    1
5    1
Name: count, dtype: int64
[0 4 1 2 5 3]
Лучшие параметры для линейной регрессии: {'fit_intercept': True}


### Оценка линейной регресии

In [5]:
# Предсказание на тестовых данных с лучшими параметрами
linear_predictions = linear_grid.predict(X_test)

# Оценка качества модели регрессии
mae = mean_absolute_error(y_test, linear_predictions)
mse = mean_squared_error(y_test, linear_predictions)
r2 = r2_score(y_test, linear_predictions)

print("MAE: ", mae)
print("MSE: ", mse)
print("R^2: ", r2)

# Округление предсказанных значений до целых чисел (поскольку это классы)
rounded_predictions = [round(pred) for pred in linear_predictions]
# Вывод отчета классификации
print(classification_report(y_test, rounded_predictions, zero_division=1))

MAE:  2.158600967335057
MSE:  6.978115917173955
R^2:  -1.400355582596124
              precision    recall  f1-score   support

         -18       0.00      1.00      0.00         0
         -13       0.00      1.00      0.00         0
         -10       0.00      1.00      0.00         0
          -9       0.00      1.00      0.00         0
          -8       0.00      1.00      0.00         0
          -7       0.00      1.00      0.00         0
          -5       0.00      1.00      0.00         0
          -4       0.00      1.00      0.00         0
          -3       0.00      1.00      0.00         0
          -2       0.00      1.00      0.00         0
          -1       0.00      1.00      0.00         0
           0       0.74      0.11      0.20      3050
           1       0.03      0.35      0.05        62
           2       0.02      0.27      0.03        60
           3       0.00      1.00      0.00         1
           4       0.27      0.11      0.15       915
        

### Класификация с использованием логической регресии


In [6]:
X = Dataset[['BLDG_TYPE_ENC', 'LIVING_AREA', 'TOTAL_VALUE', 'CITY_ENC']]
y = Dataset['COND_ENC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

logistic_model = LogisticRegression(multi_class='auto', solver='liblinear', max_iter=15000)

logistic_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}

logistic_grid = GridSearchCV(logistic_model, logistic_params, cv=5)
logistic_grid.fit(X_train, y_train)

print("Лучшие параметры для логистической регрессии:", logistic_grid.best_params_)

best_logistic_model = LogisticRegression(
    multi_class='auto',
    solver='liblinear',
    max_iter=15000,
    C=logistic_grid.best_params_['C'],
    penalty=logistic_grid.best_params_['penalty']
)

best_logistic_model.fit(X_train, y_train)

logistic_predictions = best_logistic_model.predict(X_test)
logistic_classification_report = classification_report(y_test, logistic_predictions)
print("Отчет о классификации для логистической регрессии:\n", logistic_classification_report)

logistic_probabilities = best_logistic_model.predict_proba(X_test)

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test == i, logistic_probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

Лучшие параметры для логистической регрессии: {'C': 0.001, 'penalty': 'l2'}
Отчет о классификации для логистической регрессии:
               precision    recall  f1-score   support

           0       0.74      1.00      0.85      3050
           1       0.58      0.11      0.19        62
           2       0.00      0.00      0.00        60
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00       915
           5       0.00      0.00      0.00        41

    accuracy                           0.74      4129
   macro avg       0.22      0.19      0.17      4129
weighted avg       0.56      0.74      0.63      4129

